# Comparation angles PML,DML axe de rotation

In [162]:
import os
import vtk
import numpy as np

In [ ]:
if not(os.path.exists(os.path.join(os.getcwd(),'Visualization'))):

    folder_path = os.path.join(os.getcwd(),'ANON')
    patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
    patients.sort(key=lambda x: int(x))
    legs=['Left','Right']
    os.mkdir(os.path.join(os.getcwd(),'Visualization'))
    for p in range(len(patients)):
        os.mkdir(os.path.join(os.getcwd(),'Visualization',patients[p]))
        for l in range(len(legs)): 
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
                os.mkdir(os.path.join(os.getcwd(),'Visualization',patients[p],legs[l])) 

In [164]:
with open(os.path.join(os.getcwd(),'Visualization','Right_PML_DistalPlane.txt'), 'w') as f:
    f.write("Angle Between PML and Primary Axis in the distal plane:")  
    f.write("\n(Sign positive if PML is counterclockwise  to the Primary Axis in the Proximal direction)\n")
    
with open(os.path.join(os.getcwd(),'Visualization','Right_DML_PosteriorPlane.txt'), 'w') as f:
    f.write("Angle Between DML and Primary Axis in the posterior plane:")
    f.write("\n(Sign positive if DML is counterclockwise  to the Primary Axis in the Anterior direction)\n")
    
with open(os.path.join(os.getcwd(),'Visualization','Left_PML_DistalPlane.txt'), 'w') as f:
    f.write("Angle Between PML and Primary Axis in the distal plane:")  
    f.write("\n(Sign positive if PML is counterclockwise  to the Primary Axis in the Proximal direction)\n")
    
with open(os.path.join(os.getcwd(),'Visualization','Left_DML_PosteriorPlane.txt'), 'w') as f:
    f.write("Angle Between DML and Primary Axis in the posterior plane:")
    f.write("\n(Sign positive if DML is counterclockwise  to the Primary Axis in the Anterior direction)\n")
    
with open(os.path.join(os.getcwd(),'Visualization','All_PML_DistalPlane.txt'), 'w') as f:
    f.write("Absolute Angle Between PML and Primary Axis in the distal plane:\n")  
    
with open(os.path.join(os.getcwd(),'Visualization','All_DML_PosteriorPlane.txt'), 'w') as f:
    f.write("Absolute Angle Between DML and Primary Axis in the posterior plane:\n")

In [165]:
#Reading the resection plane Normal

def ReadNormalPlane(route):
    with open(route, 'r') as f:
        contenido = f.read()
        palabras = contenido.split()
        i = palabras.index('NORMALS')
        normalc=[]
        normalc.append(palabras[i+3])
        normalc.append(palabras[i+4])
        normalc.append(palabras[i+5])
        normal=[float(char) for char in normalc]
    return normal

In [166]:
def CopyVTKFile(mesh,path):
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(path)
    writer.SetInputData(mesh)
    writer.Write()
    return

In [167]:
def LoadMesh(filename):
    reader = vtk.vtkPolyDataReader() 
    reader.SetFileName(filename)
    reader.ReadAllScalarsOn()
    reader.ReadAllVectorsOn()
    reader.Update()
    mesh = reader.GetOutput()  
    return mesh

def LoadMeshstl(filename):
    reader = vtk.vtkSTLReader()
    reader.SetFileName(filename)
    reader.Update()
    mesh = reader.GetOutput()  
    return mesh

In [168]:
def save_vtk_line_with_translation(filename, vector, center):
    # Normalizar el vector
    vector = vector / np.linalg.norm(vector)

    # Calcular los puntos de la línea
    point1 = center
    point2 = center + 35 * vector

    # Crear el objeto vtkLineSource
    line = vtk.vtkLineSource()
    line.SetPoint1(point1)
    line.SetPoint2(point2)

    # Guardar el archivo VTK
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(filename)
    writer.SetInputConnection(line.GetOutputPort())
    writer.Write()
    return

In [ ]:
Right_PML_Analysis=[]
Left_PML_Analysis=[]
Absolute_PML_Analysis=[]

Right_DML_Analysis=[]
Left_DML_Analysis=[]
Absolute_DML_Analysis=[]


folder_p = os.path.join(os.getcwd(),'ANON') #Folders with the patients
folder_path = os.path.join(os.getcwd(),folder_p)
patients = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and len(f) == 3 and f.isnumeric()]
patients.sort(key=lambda x: int(x))
legs=['Left','Right']
for p in range(len(patients)):
    for l in range(len(legs)): 
        if os.path.exists(os.path.join(folder_path,patients[p],legs[l])):
            if os.path.exists(os.path.join(folder_path,patients[p],legs[l],'ANON','ANON','ANON')):
                
                #Make a ANON copy for visualization
                if os.path.exists(os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'segmentation','ANON.stl')):
                    ANONModelRoute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'segmentation','ANON.stl')
                    ANONModel=LoadMeshstl(os.path.join(ANONModelRoute))
                    CopyVTKFile(ANONModel,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ANON.vtk'))

                #Copy the rotation axis
                RotationAxisRoute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','ax_Primary.vtk')
                RotationAxis=LoadMesh(os.path.join(RotationAxisRoute))
                CopyVTKFile(RotationAxis,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ax_Primary.vtk'))
                
                #Copy the DML
                DMLroute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','ax_Distal mediolateral.vtk')
                DML=LoadMesh(os.path.join(DMLroute))
                CopyVTKFile(DML,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ax_Distal mediolateral.vtk'))
                
                #Copy the PML
                PMLroute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','ax_Posterior mediolateral.vtk')
                PML=LoadMesh(os.path.join(PMLroute))
                CopyVTKFile(PML,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ax_Posterior mediolateral.vtk'))
                
                #Copy the Posterior resection plane
                PosteriorResectionroute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','pl_Posterior resection.vtk')
                PosteriorResection=LoadMesh(os.path.join(PosteriorResectionroute))
                CopyVTKFile(PosteriorResection,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'pl_Posterior resection.vtk'))
                
                #Copy the Distal resection plane
                DistalResectionroute=os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','pl_Distal resection.vtk')
                DistalResection=LoadMesh(os.path.join(DistalResectionroute))
                CopyVTKFile(DistalResection,os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'pl_Distal resection.vtk'))
                
                #Obtain the useful data of each
                #Normals of planes
                DistalResectionCenter=np.array(DistalResection.GetCenter())
                PosteriorResectionCenter=np.array(PosteriorResection.GetCenter())
                
                PosteriorResection=ReadNormalPlane((os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','pl_Posterior resection.vtk')))
                DistalResection=ReadNormalPlane((os.path.join(os.getcwd(),'ANON',patients[p],legs[l],'ANON','ANON','ANON','pl_Distal resection.vtk')))
                
                
                if legs[l]=='Left':
                    DistalResection=[-x for x in DistalResection] 
                    
                    RotationAxis=-(np.array(RotationAxis.GetPoint(0)) - np.array(RotationAxis.GetPoint(1)))
                    RotationAxis=RotationAxis/ np.linalg.norm(RotationAxis)

                    PML=-(np.array(PML.GetPoint(0)) - np.array(PML.GetPoint(1)))
                    PML=PML/ np.linalg.norm(PML)
                    
                    DML=(np.array(DML.GetPoint(0)) - np.array(DML.GetPoint(1)))
                    DML=DML/ np.linalg.norm(DML)                    
                    
                else:
                    PosteriorResection=[-x for x in PosteriorResection]
                    RotationAxis=(np.array(RotationAxis.GetPoint(0)) - np.array(RotationAxis.GetPoint(1)))
                    RotationAxis=RotationAxis/ np.linalg.norm(RotationAxis)
                    
                    PML=(np.array(PML.GetPoint(0)) - np.array(PML.GetPoint(1)))
                    PML=PML/ np.linalg.norm(PML)

                    DML=-(np.array(DML.GetPoint(0)) - np.array(DML.GetPoint(1)))
                    DML=DML/ np.linalg.norm(DML)    
                ###################################################################
                
                #Rotation Axis and PML in Distal Resection
                DistalResection=np.array(DistalResection)
                
                # Project Rotation Axis
                ProjectedRotationAxis = RotationAxis - np.dot(RotationAxis, DistalResection) * DistalResection
                ProjectedRotationAxis = ProjectedRotationAxis / np.linalg.norm(ProjectedRotationAxis)

                # Project PML
                ProjectedPML = PML - np.dot(PML, DistalResection) * DistalResection
                ProjectedPML = ProjectedPML / np.linalg.norm(ProjectedPML)
                
                #Calculate the angle
                cross_product = np.cross(ProjectedRotationAxis,ProjectedPML)
                Angle = np.arctan2(np.linalg.norm(cross_product), np.dot( ProjectedRotationAxis,ProjectedPML))
                Angle = np.degrees(Angle)
                cross_product= cross_product / np.linalg.norm(cross_product)
                sign = np.sign(np.dot(cross_product, np.array(DistalResection))) 
                Angle= Angle * sign
                
                #Save the Visualization
                save_vtk_line_with_translation(os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ProjectedRotationAxisDistal.vtk'), ProjectedRotationAxis, DistalResectionCenter)
                save_vtk_line_with_translation(os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ProjectedPMLDistal.vtk'), ProjectedPML, DistalResectionCenter)
    

                #Open Analysis file and write the information
                if Angle>=0:
                    space=' '
                else: space=''
        
                if legs[l]=='Right':
                    with open(os.path.join(os.getcwd(),'Visualization','Right_PML_DistalPlane.txt'), 'a') as f:
                        f.write(patients[p]+' '+legs[l]+": "+space+str(round(Angle,1))+"º\n") 
                else:
                    with open(os.path.join(os.getcwd(),'Visualization','Left_PML_DistalPlane.txt'), 'a') as f:
                        f.write(patients[p]+' '+legs[l]+": "+space+str(round(Angle,1))+"º\n")                 
                
                if legs[l]=='Left':
                    space=' '
                else:
                    space=''
                    
                with open(os.path.join(os.getcwd(),'Visualization','All_PML_DistalPlane.txt'), 'a') as f:
                    f.write(patients[p]+' '+legs[l]+": "+space+str(round(abs(Angle),1))+"º\n")  
                    
                #Save for Analysis
                if legs[l]=='Right':
                    Right_PML_Analysis.append(Angle)
                else:
                    Left_PML_Analysis.append(Angle)
                
                Absolute_PML_Analysis.append(abs(Angle))
                ###################################################################
                
                #Rotation Axis and DML in Posterior Resection
                PosteriorResection=np.array(PosteriorResection)
                
                # Project Rotation Axis
                ProjectedRotationAxis = RotationAxis - np.dot(RotationAxis, PosteriorResection) * PosteriorResection
                ProjectedRotationAxis = ProjectedRotationAxis / np.linalg.norm(ProjectedRotationAxis)

                # Project DML
                ProjectedDML = DML - np.dot(DML, PosteriorResection) * PosteriorResection
                ProjectedDML = ProjectedDML / np.linalg.norm(ProjectedDML)
                
                #Calculate the angle
                cross_product = np.cross(ProjectedRotationAxis,ProjectedDML)
                Angle = np.arctan2(np.linalg.norm(cross_product), np.dot( ProjectedRotationAxis,ProjectedDML))
                Angle = np.degrees(Angle)
                cross_product= cross_product / np.linalg.norm(cross_product)
                sign = np.sign(np.dot(cross_product, np.array(PosteriorResection))) 
                Angle= Angle * sign
                
                #Save the Visualization
                save_vtk_line_with_translation(os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ProjectedRotationAxisPosterior.vtk'), ProjectedRotationAxis, PosteriorResectionCenter)
                save_vtk_line_with_translation(os.path.join(os.getcwd(),'Visualization',patients[p],legs[l],'ProjectedDMLPosterior.vtk'), ProjectedDML, PosteriorResectionCenter)
                    
                #Open Analysis file and write the information
                if Angle>=0:
                    space=' '
                else: space=''
                    
                if legs[l]=='Right':
                    with open(os.path.join(os.getcwd(),'Visualization','Right_DML_PosteriorPlane.txt'), 'a') as f:
                        f.write(patients[p]+' '+legs[l]+": "+space+str(round(Angle,1))+"º\n") 
                else:
                    with open(os.path.join(os.getcwd(),'Visualization','Left_DML_PosteriorPlane.txt'), 'a') as f:
                        f.write(patients[p]+' '+legs[l]+": "+space+str(round(Angle,1))+"º\n")       
                        
                if legs[l]=='Left':
                    space=' '
                else:
                    space=''
                    
                with open(os.path.join(os.getcwd(),'Visualization','All_DML_PosteriorPlane.txt'), 'a') as f:
                    f.write(patients[p]+' '+legs[l]+space+": "+str(round(abs(Angle),1))+"º\n")  
                
                #Save for Analysis
                if legs[l]=='Right':
                    Right_DML_Analysis.append(Angle)
                else:
                    Left_DML_Analysis.append(Angle)
                
                Absolute_DML_Analysis.append(abs(Angle))

In [170]:
#Analysis
Right_PML_Analysis=np.array(Right_PML_Analysis)
Right_Mean_PML = np.mean(Right_PML_Analysis)
Right_SD_PML =np.std(Right_PML_Analysis)
Right_Max_PML = np.max(Right_PML_Analysis)
Right_Min_PML = np.min(Right_PML_Analysis)

Right_DML_Analysis=np.array(DML_Analysis) 
Right_Mean_DML = np.mean(Right_DML_Analysis)
Right_SD_DML =np.std(Right_DML_Analysis)
Right_Max_DML = np.max(Right_DML_Analysis)
Right_Min_DML = np.min(Right_DML_Analysis)


Left_PML_Analysis=np.array(Left_PML_Analysis)
Left_Mean_PML = np.mean(Left_PML_Analysis)
Left_SD_PML =np.std(Left_PML_Analysis)
Left_Max_PML = np.max(Left_PML_Analysis)
Left_Min_PML = np.min(Left_PML_Analysis)

Left_DML_Analysis=np.array(Left_DML_Analysis) 
Left_Mean_DML = np.mean(Left_DML_Analysis)
Left_SD_DML =np.std(Left_DML_Analysis)
Left_Max_DML = np.max(Left_DML_Analysis)
Left_Min_DML = np.min(Left_DML_Analysis)

Absolute_PML_Analysis=np.array(Absolute_PML_Analysis)
Absolute_Mean_PML = np.mean(Absolute_PML_Analysis)
Absolute_SD_PML =np.std(Absolute_PML_Analysis)
Absolute_Max_PML = np.max(Absolute_PML_Analysis)
Absolute_Min_PML = np.min(Absolute_PML_Analysis)

Absolute_DML_Analysis=np.array(Absolute_DML_Analysis) 
Absolute_Mean_DML = np.mean(Absolute_DML_Analysis)
Absolute_SD_DML =np.std(Absolute_DML_Analysis)
Absolute_Max_DML = np.max(Absolute_DML_Analysis)
Absolute_Min_DML = np.min(Absolute_DML_Analysis)

with open(os.path.join(os.getcwd(),'Visualization','Left_Analysis.txt'), 'w') as f:
    f.write("Angle Between PML and Primary Axis in the distal plane (º):")  
    f.write("\n(Sign positive if PML is counterclockwise  to the Primary Axis in the Proximal direction)\n")  
    f.write(str(round(Left_Mean_PML,1))+" \u00B1 "+str(round(Left_SD_PML,1))+" ("+str(round(Left_Min_PML,1))+" to " +str(round(Left_Max_PML,1))+")")
    f.write("\n\nAngle Between DML and Primary Axis in the posterior plane (º):")
    f.write("\n(Sign positive if DML is counterclockwise  to the Primary Axis in the Anterior direction)\n")          
    f.write(str(round(Left_Mean_DML,1))+" \u00B1 "+str(round(Left_SD_DML,1))+" ("+str(round(Left_Min_DML,1))+" to " +str(round(Left_Max_DML,1))+")")
    
with open(os.path.join(os.getcwd(),'Visualization','Right_Analysis.txt'), 'w') as f:
    f.write("Angle Between PML and Primary Axis in the distal plane (º):")  
    f.write("\n(Sign positive if PML is counterclockwise  to the Primary Axis in the Proximal direction)\n")  
    f.write(str(round(Right_Mean_PML,1))+" \u00B1 "+str(round(Right_SD_PML,1))+" ("+str(round(Right_Min_PML,1))+" to " +str(round(Right_Max_PML,1))+")")
    f.write("\n\nAngle Between DML and Primary Axis in the posterior plane (º):")
    f.write("\n(Sign positive if DML is counterclockwise  to the Primary Axis in the Anterior direction)\n")          
    f.write(str(round(Right_Mean_DML,1))+" \u00B1 "+str(round(Right_SD_DML,1))+" ("+str(round(Right_Min_DML,1))+" to " +str(round(Right_Max_DML,1))+")")
    
with open(os.path.join(os.getcwd(),'Visualization','All_Analysis.txt'), 'w') as f:
    f.write("Absolut Angle Between PML and Primary Axis in the distal plane (º):\n")  
    f.write(str(round(Absolute_Mean_PML,1))+" \u00B1 "+str(round(Absolute_SD_PML,1))+" ("+str(round(Absolute_Min_PML,1))+" to " +str(round(Absolute_Max_PML,1))+")")
    f.write("\n\nAbsolut Angle Between DML and Primary Axis in the posterior plane (º):\n")         
    f.write(str(round(Absolute_Mean_DML,1))+" \u00B1 "+str(round(Absolute_SD_DML,1))+" ("+str(round(Absolute_Min_DML,1))+" to " +str(round(Absolute_Max_DML,1))+")")